<center> </center>

<center><font size=5 face="Helvetica" color=#EE4B2B><b>
Pytorch Tutorial: Build the Neural Network
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>July, 2024</font></center>

***

Neural networks comprise of layers/modules that perform operations on data.
The $\texttt{torch.nn}$ namespace provides all the building blocks you need to build your neural neetwork.
Every module in PyTorch subclasses the $\mathtt{nn.Module}$.
A neural network is a module itself that consists of other modules (layers).
This nested structure allows for building and managing complex architectures easily.

In the fowllowing sections, we will build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available.
Let's check to see if $\texttt{torch.cuda}$ or $\texttt{torch.backend.mps}$ are available, otherwise we use the CPU.

In [2]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
print(f"Using {device} device.")

Using mps device.


# Define the Class

We define our neural network by subclassing $\texttt{nn.Module}$, and initialize the neural network layers in $\texttt{\_\_init\_\_}$.
Every $\texttt{nn.Module}$ subclass implements the operations on input data in the $\texttt{forward}$ method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of $\texttt{NeuralNetwork}$, and move it to the $\texttt{device}$, and print its structure.

In [5]:
model = NeuralNetwork().to(device=device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

To use the model. we pass it the input data.
This executes the model's $\texttt{forward}$, along with some background operations.
Do not call $\texttt{model.forward()}$ directly.

Calling the model on the imput returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instantce of $\texttt{nn.Softmax}$ module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='mps:0')


# Model Layers

Let's break down the layers in the FashionMNIST model.
To illustrate it, we will take a sample minibatch of 3 images of size 28x28 
and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
input_image.size()

torch.Size([3, 28, 28])

## nn.Flatten

We initialize the $\texttt{nn.Flatten}$ layer to convert each 2D 28$\times$28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

## nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

## nn.ReLU

Non-linear activations are what create the complex mapping between the model's inputs and outputs.
They are apllied after linear transformations to introduc nonlinearity,
helping neural neetworks learn a wide variety of phenomena.

In this model, we use $\texttt{nn.ReLU}$ bewtween our linear layers, but there's other activatoins to introduce non-linearity in our model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}\n\n")

Before ReLU: tensor([[-4.2737e-01, -4.9111e-01,  3.4963e-01,  5.6632e-01,  2.2415e-01,
          1.1379e-01,  9.7172e-03,  3.8196e-01,  4.2302e-02,  3.1843e-01,
         -1.2421e-01, -8.9374e-01, -1.8459e-02, -1.9446e-01,  1.5748e-02,
          7.8721e-04, -1.5392e-01, -3.9268e-02,  9.2933e-02, -2.1698e-01],
        [-1.9865e-01, -2.2511e-01,  8.4534e-02,  3.6073e-01,  2.9812e-01,
         -1.4895e-01, -3.8197e-02,  2.5157e-01, -9.6405e-04,  2.0031e-01,
         -2.1452e-01, -5.9686e-01, -1.9009e-01, -2.4979e-01, -4.1683e-01,
         -3.5008e-01, -6.3378e-03,  3.0951e-01, -1.1961e-03, -1.2958e-01],
        [-2.6814e-01, -4.6044e-01,  2.1683e-01,  3.7727e-01, -6.6986e-02,
          6.2326e-02,  7.0886e-03,  2.7343e-01,  2.9366e-01,  7.0789e-02,
         -4.0734e-01, -6.9680e-01, -3.6037e-01, -1.7712e-01, -5.1263e-01,
         -1.9949e-01, -1.2035e-01,  7.8629e-02, -2.4945e-01, -1.6903e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.3496, 0.5663, 0.2241,

## nn.Sequential

$\texttt{nn.Sequential}$ is an ordered container of modules. 
The data is passed through all the modules in the same order as defined. 
You can use sequential containers to put together a quick network like $\texttt{seq\_modules}$.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[ 0.0651, -0.0931,  0.1512,  0.1764, -0.0966,  0.0848, -0.0235,  0.1350,
         -0.1150, -0.1079],
        [-0.0262, -0.1801,  0.0466,  0.0065, -0.0154,  0.1601, -0.0083,  0.0682,
         -0.2406,  0.0362],
        [ 0.0985, -0.0745,  0.0929,  0.1635, -0.1037,  0.0705,  0.0041,  0.0300,
         -0.0307, -0.1618]], grad_fn=<AddmmBackward0>)

## nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. 
The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class.
$\texttt{dim}$ parameter indicates the dimension along which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1042, 0.0890, 0.1136, 0.1165, 0.0887, 0.1063, 0.0954, 0.1118, 0.0870,
         0.0877],
        [0.0983, 0.0843, 0.1057, 0.1016, 0.0994, 0.1185, 0.1001, 0.1081, 0.0794,
         0.1047],
        [0.1089, 0.0916, 0.1083, 0.1162, 0.0889, 0.1059, 0.0991, 0.1017, 0.0957,
         0.0839]], grad_fn=<SoftmaxBackward0>)

In [13]:
torch.sum(pred_probab[2,:])

tensor(1.0000, grad_fn=<SumBackward0>)

# Model Parameters

Many layers inside a neural network are *parameterized*, i.e. have associated weights and biases that are optimized during training.
Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's $\texttt{parameters()}$ or $\texttt{named\_parameters()}$ methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0315, -0.0311,  0.0135,  ...,  0.0152, -0.0211, -0.0012],
        [-0.0018, -0.0058, -0.0242,  ..., -0.0053,  0.0262, -0.0304]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0096, -0.0253], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0414, -0.0326,  0.0441,  ...,  0.0087,  0.0306, -0.0407],
        [-0.0062, -0.0224,  0.0008,  ..., -0.0162, -0.0333,  0.0048]],
       device='mps:0', grad_fn=<Slice